In [9]:
import pandas as pd
import math
documents = [
    ['antony', 'brutus', 'caeser', 'cleopatra', 'mercy', 'worser'],
    ['antony', 'brutus', 'caeser', 'calpurnia'],
    ['mercy', 'worser'],
    ['brutus', 'caeser', 'mercy', 'worser'],
    ['caeser', 'mercy', 'worser'],
    ['antony', 'caeser', 'mercy'],
    ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
    ['angels', 'fools', 'fear', 'in', 'rush', 'to', 'tread', 'where'],
    ['angels', 'fools', 'in', 'rush', 'to', 'tread', 'where'],
    ['fools', 'fear', 'in', 'rush', 'to', 'tread', 'where']
    ]

all_words = []
for doc in documents:
    for word in doc:
        all_words.append(word)

def get_term_freq(doc):
    words_found = dict.fromkeys(all_words, 0)
    for word in doc:
        words_found[word] += 1
    return words_found

term_freq = pd.DataFrame(get_term_freq(documents[0]).values(), index=get_term_freq(documents[0]).keys())

for i in range(1, len(documents)):
    term_freq[i] = get_term_freq(documents[i]).values()

term_freq.columns = ['doc'+str(i) for i in range(1, 11)]

def get_weighted_term_freq(x):
    if x > 0:
        return math.log10(x) + 1
    return 0

for i in range(1, len(documents)+1):
    term_freq['doc'+str(i)] = term_freq['doc'+str(i)].apply(get_weighted_term_freq)


tfd = pd.DataFrame(columns=['freq', 'idf'])

for i in range(len(term_freq)):

    frequency = term_freq.iloc[i].values.sum()

    tfd.loc[i, 'freq'] = frequency

    tfd.loc[i, 'idf'] = math.log10(10 / (float(frequency)))

tfd.index = term_freq.index



term_freq_inve_doc_freq = term_freq.multiply(tfd['idf'], axis=0)

import numpy as np
document_length = pd.DataFrame()

def get_docs_length(col):
    return np.sqrt(term_freq_inve_doc_freq[col].apply(lambda x: x**2).sum())

for column in term_freq_inve_doc_freq.columns:
    document_length.loc[0, column+'_len'] = get_docs_length(column)



normalized_term_freq_idf = pd.DataFrame()

def get_normalized(col, x):
    try:
        return x / document_length[col+'_len'].values[0]
    except:
        return 0

for column in term_freq_inve_doc_freq.columns:
    normalized_term_freq_idf[column] = term_freq_inve_doc_freq[column].apply(lambda x : get_normalized(column, x))


normalized_term_freq_idf

,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10
antony,0.380701,0.408621,0.000000,0.000000,0.000000,0.775474,0.000000,0.000000,0.000000,0.000000
brutus,0.380701,0.408621,0.000000,0.667839,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
caeser,0.219176,0.235250,0.000000,0.384486,0.516570,0.446453,0.000000,0.000000,0.000000,0.000000
cleopatra,0.728087,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
mercy,0.219176,0.000000,0.603298,0.384486,0.516570,0.446453,0.000000,0.000000,0.000000,0.000000
worser,0.289735,0.000000,0.797516,0.508263,0.682869,0.000000,0.000000,0.000000,0.000000,0.000000
calpurnia,0.000000,0.781483,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
angels,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.472707,0.000000
fools,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.325248,0.325248,0.359756,0.359756
fear,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.427365,0.427365,0.000000,0.472707


In [12]:
q= 'antony brutus'


In [35]:
def get_w_tf (x):
    try:
        return math.log10(x)+1
    except:
        return 0

In [37]:
query = pd.DataFrame(index=normalized_term_freq_idf.index)
query['tf'] = [ 1 if x in q.split() else 0 for x in list(normalized_term_freq_idf.index)]
query['w_tf'] = query['tf'].apply(lambda x :get_w_tf(x))
product=normalized_term_freq_idf.multiply(query['w_tf'],axis=0)
query['idf']=tfd['idf']*query['w_tf']
query['norm'] =0
for i in range(len(query)):
    query['norm'].iloc[i]=float(query['idf'].iloc[i])/math.sqrt(sum(query['idf'].values**2))
product2= product.multiply(query['norm'],axis=0)
product2    

C:\Users\Dell\AppData\Local\Temp\ipykernel_9592\250365627.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query['norm'].iloc[i]=float(query['idf'].iloc[i])/math.sqrt(sum(query['idf'].values**2))


,doc1,doc2,doc3,doc4,doc5,doc6,doc7,doc8,doc9,doc10
antony,0.269196,0.288939,0.0,0.000000,0.0,0.548343,0.0,0.0,0.0,0.0
brutus,0.269196,0.288939,0.0,0.472234,0.0,0.000000,0.0,0.0,0.0,0.0
caeser,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
cleopatra,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
mercy,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
worser,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
calpurnia,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
angels,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
fools,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0
fear,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0


In [44]:
# query length
math.sqrt(sum([x**2 for x in query['idf'].loc[q.split()]]))

0.7394622130520805

In [ ]:
product2.loc[q.split()].values

In [41]:
scores={}
for col in product2.columns:
    if 0 in product2[col].loc[q.split()].values:
        pass
    else:
        scores[col]= product2[col].sum()


In [42]:
scores

{'doc1': 0.5383927937463102, 'doc2': 0.5778771030041435}

In [50]:
# doc1 doc2
product2[list(scores.keys())].loc[q.split()]

,doc1,doc2
antony,0.269196,0.288939
brutus,0.269196,0.288939


In [51]:
prod_res= product2[list(scores.keys())].loc[q.split()]

In [52]:
#sum == cosine similarity (q , doc)
prod_res.sum()

doc1    0.538393
doc2    0.577877
dtype: float64

In [56]:
# returned docs
ranked =sorted(scores.items(),key=lambda x : x[1], reverse=True)

In [58]:
for doc in ranked:
    print(doc[0],end=' ')

doc2 doc1 